# Genomics to Notebook (g2nb): Enabling cross-platform compatibility in Jupyter notebooks

The code below may be used to reproduce the results of the Genomics to Notebook (g2nb) paper.

<h2>Install the g2nb Extensions</h2>
<p>Run the cell below to install the g2nb extensions. You may need to refresh your browser afterward before the extensions are loaded in your browser.</p>

In [ ]:
!pip install g2nb

<h2>Import the Libraries</h2>
<p>The code below imports the <a href="https://docs.g2nb.org/en/latest/programmatic/#ui-builder" target="_blank" rel="noopener">User Interface Builder</a> decorator, which is used to a create graphical user interface. It also imports other libraries used in the code examples that follow.</p>

In [ ]:
from nbtools import build_ui, UIBuilder
from IPython.display import display
from urllib.request import urlopen

import builtins
import json
import re

<h2>Create a User Interface Builder Tool</h2>
<p>The code below employs the User Interface Builder to create a user-friendly graphical widget. This widget accepts a CX file and launches cytoscape.js with a view of the network.</p> 
<p>If you need a CX file to use as an example, you may download <a href="https://github.com/cytoscape/cy-jupyterlab/blob/master/example/example.cx">the one linked here</a>. For the CX file to work correctly, ensure that you download the raw file.</p>

In [ ]:
@build_ui(name='Cytoscape Network', 
          description='Displays a Cytoscape network in the notebook', 
          origin='+', color='rgb(210, 140, 34)',
          parameters={
              'cx_file': {
                  'required': True,
                  'description': 'Select a Cyoscape network to view',
                  'type': 'file',
                  'kinds': ['cx']
              }
          })
def dispplay_cytoscape_network(cx_file=''):
    with open(cx_file) as f: data = f.read()
    data_json = json.loads(data)
    cx_data = { "application/cx": data_json }
    display(cx_data, raw=True)

## Expanded Version

An expanded version of the code that handles remote URLs as well as local files is included for reference below.

In [ ]:
@build_ui(name='Cytoscape Network With URL Support',
          description='Displays a Cytoscape network in the notebook.',
          origin='+', color='rgb(210, 140, 34)',
          parameters={
              'cx_file': {
                  'required': True,
                  'description': 'Select a Cytoscape network to view',
                  'type': 'file',
                  'kinds': ['cx']
              }
          })
def display_cytoscape_network(cx_file=''):
    # Determine if the input is a file path or URL
    is_url = re.compile(
        r'^(?:http|ftp)s?://'  # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|'  # domain...
        r'localhost|'  # localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})'  # ...or ip
        r'(?::\d+)?'  # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)

    # Set the opener based on local or remote
    if re.match(is_url, cx_file): open_file = urlopen
    else: open_file = builtins.open

    # Read the contents of the file
    data = ''
    with open_file(cx_file) as f: data = f.read()

    # Display the network
    data_json = json.loads(data)
    cx_data = { "application/cx": data_json }
    display(cx_data, raw=True)

## User Interface Builder Features

Below are a selection of User Interface Builder features may be useful when integrating a bioinformatics tool.

<h3 id="Python-Variables-&amp;-String-Literals">Python Variables &amp; String Literals</h3>
<p>Python variables may be used as input when filling out a UI Builder form. To do this, simply type the name of the variable into the input field. When the form is submitted, the widget will pass a reference to the variable in the resulting function call.</p>
<p>Conversely, to ensure that an input value is evaluated as a string literal rather than a variable name, a user can wrap the input to a particular field in either single or double quotes (' or "). This tells the UI Builder to skip checking for variable names and to treat the value in quotes as a literal string. For example, forcing the string <em>foo</em> to be treated as a string literal would be entered in the input field as: <code>"foo"</code>.</p>

In [ ]:
foo = "This is a Python variable"

In [ ]:
@build_ui
def variable_example(input_text='foo'):
    """Print what was input"""
    print(input_text)

<h3 id="Rendering-Existing-Functions">Rendering Existing Functions</h3>
<p>Existing Python functions, such as those included in third-party Python libraries, can also be used with the UI Builder. To display an existing function, first import it and then pass the function into the constructor of a <code>UIBuilder</code> object. Return this object in a cell to display the resulting widget. For example, the code for displaying the <code>os</code> module's <code>chdir</code> function is given below.</p>

In [ ]:
from os import chdir

UIBuilder(chdir)

<h3 id="Hiding-Parameters">Hiding Parameters</h3>
<p>Sometimes a particular function has parameters that shouldn't be changed in the current context or which the notebook author does not wish to expose. The UI Builder has the ability to hide the input for these parameters, simplifying the user interface and allowing users to focus only on the relevant inputs.</p>
<p>When the function is called, these hidden parameters will automatically use their default values. This may be combined with overriding the default value for the parameter in question in order to force a particular input.</p>
<p>An example is given below in which a parameter is hidden.</p>

In [ ]:
@build_ui(parameters={
    "param_2": { 
        "hide": True, 
        "default": "Default value" 
    }
})
def hiding_example(param_1, param_2):
    """Hide parameters"""
    print(param_2)

<h3 id="Output-Variable">Output Variable</h3>
<p>The result of a UI Builder function can optionally be assigned to a Python variable. If enabled, a text field for this variable will appear at the bottom of a UI Builder widget. This field can be overridden just like any other parameter using the <code>output_var</code> parameter name. An example is given below. Enter a text value into the input form, then note that this value is assigned to the `results` variable after the function is executed.</p>

In [ ]:
@build_ui(parameters={
    "output_var": {
        "name": "output",
        "default": "results",
        "description": "The results of the function",
        "hide": False,
    }
})
def output_var_example(param_1):
    """Override and hide the output variable"""
    return param_1

In [ ]:
results

<h3 id="Parameter-Types">Parameter Types</h3>
<p>The UI Builder supports a number of parameter types and implements features to make handling those types easier. Supported types include:</p>
<ul>
<li><strong>text:</strong> Supports any text value. Text is also the default parameter type if no other type information has been specified or can be determined.</li>
<li><strong>number:</strong> Accepts any numerical value and renders itself in a notebook as an HTML number input.</li>
<li><strong>password:</strong> Works exactly like a text input, but obfuscates the input value as a password field.</li>
<li><strong>choice:</strong> When provided with a list of choices, this input will render as a dropdown parameter, with the default value selected. Choice parameters are described in their own section below.</li>
<li><strong>bool:</strong> A boolean input representing True and False. Renders as a choice parameter with those two options.</li>
<li><strong>file:</strong> An input intended to receive a file or file-like object. File parameters are described in their own section below.</li>
</ul>
<p>The UI Builder will infer a parameter's type from its default value, defaulting to a text parameter if no value is available or if the default value's type doesn't match one of the known types above. Alternatively,you may specify a parameter's type in the code. An example is provided below. It illustrates how to specify each type, except for choice and file parameters, which are each detailed in their own sections.</p>

In [ ]:
@build_ui(parameters={
    "a_text_param": {
        "type": "text"
    },
    "a_number_param": {
        "type": "number"
    },
    "a_password_param": {
        "type": "password"
    },
    "a_bool_param": {
        "type": "bool"
    },
})
def type_example(a_text_param, a_number_param, a_password_param, a_bool_param=True):
    """Parameter type example"""
    print([a_text_param, a_number_param, a_password_param, a_bool_param])

<h4 id="Choice-Parameters">Choice Parameters</h4>
<p>Sometimes a parameter only accepts a limited set of valid input values. This is often represented in a user interface as a dropdown (select) input. The UI Builder has support for this functionality. To change a particular parameter into a dropdown input, simply set the type to "choice" and provide the parameter with a dictionary of available choices.</p>

In [ ]:
@build_ui(parameters={
    "param_1": {
        "default": "some_value",
        "type": "choice",
        "choices": {
            "foo label": "foo value",
            "bar label": "bar value",
            "some_label": "some_value"
        }
    } 
})
def choice_example(param_1):
    """Select a value from a dropdown"""
    print(param_1)

<h4 id="File-Parameters">File Parameters</h4>
<p>File parameters are intended to handle input representing a file or file-like object. Optionally, you may specify the kinds of files that a file parameter accepts. This is accomplished by providing a list of file extensions. For example, a parameter that expects a gct, odf or res file would list: <code>["gct", "odf", "res"]</code>.</p>
<p>It is worth noting that when a file is selected in the menu, the value provided to the function will actually be a string containing a path or URL to the specified file. This may be used to create a file-like object.</p>

In [ ]:
@build_ui(parameters={
    "param_1": {
        "type": "file",
        "kinds": ["gct", "odf"]
    }
})
def file_example(param_1):
    """Create a file parameter"""
    print(param_1)

<p>Similar to choice parameters, a file parameter may likewise be given a dictionary of possible options. These options will appear in a dropdown when the parameter it selected.</p>

In [ ]:
@build_ui(parameters={
    "param_1": {
        "type": "file",
        "choices": {
            "Example Label #1": "ftp://fake.example.com/example_1.csv",
            "Example Label #2": "ftp://fake.example.com/example2.csv",
            "Example Label #3": "ftp://fake.example.com/example_3.csv"
        }
    }
})
def file_choice_example(param_1):
    """Create a file parameter with choices"""
    print(param_1)

<h3 id="Files-in-Markdown-Cells">Files in Markdown Cells</h3>
<p>Sometimes it is beneficial to the narrative of a notebook to link an externally-hosted file in a markdown cell and then to use that file as input to a UI Builder function. This can be made easier by annotating the external file link with the <code>nbtools-markdown-file</code> class. Doing so will make the file automatically appear as an option in file parameters.</p>

In [ ]:
%%html

<a class="nbtools-markdown-file" href="https://datasets.genepattern.org/data/all_aml/all_aml_test.cls">An annotated link to an external file.</a>

In [ ]:
@build_ui(parameters={
    "param_1": {
        "type": "file",
        "kinds": ["cls"]
    }
})
def markdown_file_example(param_1):
    """Markdown file example"""
    print(param_1)

<h3 id="Messages">Messages</h3>
<p>You can set informative messages on a UI Builder widget to display useful information to the user. These may either be specified when the widget is created or dynamically within the wrapped function, adding a degree of interactivity. Message types include:</p>
<ul>
<li><strong>info:</strong> Sets an informative message to display to the user in a highlighted info callout.</li>
<li><strong>error:</strong> Sets an error message to display to the user in an error callout.</li>
</ul>

In [ ]:
@build_ui(info='You must do something before running this function.',
                  error='An error was encountered performing the function.')
def message_example(first_parameter):
    print(first_parameter)